In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas.io import data, wb
import datetime
import os.path
import math

from pandas.tseries.offsets import BDay
import talib as ta

In [ ]:
def round_2d(price):
    return math.ceil(price*100)/100

In [41]:
symbol = 'SPY'
df_train = pd.read_csv('data/'+symbol+'-ohlc.csv', index_col='Date', parse_dates=True)
if (os.path.isfile('data/'+symbol+'-dividends.csv')):
    df_dividends = pd.read_csv('data/'+symbol+'-dividends.csv', index_col='Date', parse_dates=True)
    df_train = df_train.join(df_dividends)
    df_train['Dividends'].fillna(0, inplace=True)
else:
    df_train['Dividends']=0

In [42]:
df_train['symbol'] = symbol

high = df_train['High'].values
low = df_train['Low'].values
close = df_train['Close'].values    
atr = ta.ATR(high, low, close, timeperiod=10)
        

pma5 = df_train['Close'].rolling(window=5,center=False).mean()
pma20 = df_train['Close'].rolling(window=20,center=False).mean()
pma50 = df_train['Close'].rolling(window=50,center=False).mean()
pma200 = df_train['Close'].rolling(window=200,center=False).mean() 

prev_day_open_norm = df_train['Open'] - df_train['Open']
prev_day_high_norm = (df_train['High'] - df_train['Open'])/atr
prev_day_low_norm = (df_train['Low'] - df_train['Open'])/atr
prev_day_close_norm = (df_train['Close'] - df_train['Open'])/atr

vma100 = df_train['Volume'].rolling(window=100, center=False).mean()
prev_day_vol_norm = df_train['Volume']/vma100

df_train = df_train.assign(atr=atr)
df_train = df_train.assign(pma5=pma5)
df_train = df_train.assign(pma20=pma20)
df_train = df_train.assign(pma50=pma50)
df_train = df_train.assign(pma200=pma200)

df_train = df_train.assign(prev_day_open_norm=prev_day_open_norm)
df_train = df_train.assign(prev_day_high_norm=prev_day_high_norm)
df_train = df_train.assign(prev_day_low_norm=prev_day_low_norm)
df_train = df_train.assign(prev_day_close_norm=prev_day_close_norm)
df_train = df_train.assign(prev_day_vol_norm=prev_day_vol_norm)


df_train.rename(columns = {'Close':'prev_day_close'}, inplace = True)
    
df_train.index = df_train.index + BDay(1)
df_train.to_csv('binary-SPY.txt')